In [1]:
import torch

In [2]:
preds = torch.tensor(
    [
        [
            [0.0000e00],
            [1.7091e-04],
            [0.0000e00],
            [1.2972e-05],
            [7.4123e-05],
            [0.0000e00],
            [6.2838e-05],
            [2.1668e-05],
            [0.0000e00],
            [7.9288e-05],
            [0.0000e00],
            [0.0000e00],
            [4.1136e-05],
            [4.6410e-05],
            [0.0000e00],
            [1.6944e-05],
            [1.6733e-04],
            [0.0000e00],
            [2.7461e-05],
            [0.0000e00],
            [2.5418e-04],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [4.1384e-05],
            [8.7086e-05],
            [0.0000e00],
            [5.9463e-05],
            [0.0000e00],
            [0.0000e00],
            [4.9350e-03],
            [4.1135e-04],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [5.3831e-03],
            [7.3946e-05],
            [7.5058e-05],
            [0.0000e00],
            [0.0000e00],
            [2.8100e-05],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [5.2669e-03],
            [0.0000e00],
            [6.8879e-05],
            [0.0000e00],
            [0.0000e00],
            [3.5623e-05],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
            [1.9181e-04],
            [0.0000e00],
            [1.5686e-04],
            [0.0000e00],
            [0.0000e00],
            [0.0000e00],
        ]
    ]
)

In [3]:
preds = preds[preds != 0.0]
preds

tensor([1.7091e-04, 1.2972e-05, 7.4123e-05, 6.2838e-05, 2.1668e-05, 7.9288e-05,
        4.1136e-05, 4.6410e-05, 1.6944e-05, 1.6733e-04, 2.7461e-05, 2.5418e-04,
        4.1384e-05, 8.7086e-05, 5.9463e-05, 4.9350e-03, 4.1135e-04, 5.3831e-03,
        7.3946e-05, 7.5058e-05, 2.8100e-05, 5.2669e-03, 6.8879e-05, 3.5623e-05,
        1.9181e-04, 1.5686e-04])

In [10]:
-torch.log(preds.mean())

tensor(7.2872)

In [4]:
(-torch.log(preds)).mean()

tensor(9.1237)

## torch cross entropy experiments 

In [33]:
# 16 elements, 2 classes
logits = torch.rand(4, 2)
logits = torch.nn.Softmax(dim=1)(logits)
logits[logits < 0.5] = 0.0
logits

tensor([[0.0000, 0.5598],
        [0.0000, 0.5251],
        [0.6172, 0.0000],
        [0.6078, 0.0000]])

In [34]:
labels = torch.tensor([0, 1, 0, 1])

labels[:2] = 0
labels

tensor([0, 0, 0, 1])

In [36]:
preds = logits[torch.arange(4), labels]
preds

tensor([0.0000, 0.0000, 0.6172, 0.0000])

In [40]:
-torch.log(preds + 1e-5).mean()

tensor(8.7553)

In [35]:
loss = torch.nn.CrossEntropyLoss()
loss(logits, labels)

tensor(0.8689)

In [65]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Initialize the model
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Define a sentence
sentence = "I love to play football."

# Encode the sentence
inputs = tokenizer(sentence, return_tensors="pt")

# Get the input IDs
input_ids = inputs["input_ids"].squeeze()  # squeeze to remove the batch dimension
print(f"input_ids: {input_ids}")

# Calculate the number of 40% tokens
num_tokens_to_mask = int(len(input_ids) * 0.5)

# Choose random tokens to mask, fix seed to reproduce results
np.random.seed(42)
mask_indices = np.random.choice(len(input_ids), num_tokens_to_mask, replace=False)

# Clone the input_ids for labels
labels = input_ids.clone()

# Set the label of all non-masked tokens to -100
labels[~torch.tensor(mask_indices)] = -100

# Mask the chosen tokens in input_ids
masked_input_ids = input_ids.clone()
masked_input_ids[mask_indices] = tokenizer.mask_token_id

print(f"masked input_ids: {masked_input_ids}")
print(f"labels: {labels}")


# Forward pass through the model
outputs = model(
    input_ids=input_ids.unsqueeze(0), labels=labels.unsqueeze(0)
)  # unsqueeze to add the batch dimension

# Print the loss
print(f"Loss: {outputs.loss.item()}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


input_ids: tensor([ 101, 1045, 2293, 2000, 2377, 2374, 1012,  102])
masked input_ids: tensor([ 103,  103, 2293, 2000, 2377,  103, 1012,  103])
labels: tensor([-100, 1045, -100, 2000, 2377, 2374, -100, -100])
Loss: 0.3133973181247711


In [67]:
logits = outputs.logits
probs = torch.nn.Softmax(dim=2)(logits)
indices = torch.argmax(probs, dim=2)
indices

tensor([[1012, 1012, 2293, 2000, 2377, 2374, 1012, 1012]])

In [69]:
input_ids

tensor([ 101, 1045, 2293, 2000, 2377, 2374, 1012,  102])